Import libraries needed and get/clean dataset

In [ ]:
import pandas as pd
import requests
import json
import time
import re #regular expressions
from concurrent.futures import ThreadPoolExecutor, as_completed
import unicodedata
import os

In [ ]:
# Load the CSV file, specifying the semicolon delimiter
file_path = 'BaseDataset.csv'
df = pd.read_csv(file_path,encoding="utf-8")
df.head(5)

#clean table: if track name contains "(feat. %)" or "(with %)" then remove that text from the song title 
#using regular expressions and making sure its not case sensitive
df['track_name'] = df['track_name'].replace(r'(?i)\s*\(feat\.?[^)]*\)','', regex=True)
df['track_name'] = df['track_name'].replace(r'(?i)\s*\(with\.?[^)]*\)','', regex=True)

#fix utf-8 text issue i am seeing in the dataset. (special characters like apostrophe look to be encoded as cp1252.)
def fix_text(text):
    if isinstance(text, str):
        try:
            return text.encode('cp1252', errors='replace').decode('utf-8', errors='replace')
        except UnicodeDecodeError:
            #wasn't working before when I tried to 
            return text.encode('cp1252', errors='replace').decode('utf-8', errors='ignore')
    return text

#to match hot100, it doesnt have the spanish alphabet characters. the code below should normalize the song titles to match the database
def remove_accents(text):
    text = str(text) #some songs are numbers (:
    # Normalize the text to decompose accented characters
    normalized_text = unicodedata.normalize('NFKD', text)
    # Remove the combining characters (accents) by filtering them out
    return "".join(c for c in normalized_text if not unicodedata.combining(c))

df['track_name'] = df['track_name'].apply(fix_text)
df['artists'] = df['artists'].apply(fix_text)

df['track_name'] = df['track_name'].str.replace("’", "'", regex=False)
df['artists'] = df['artists'].str.replace("’", "'", regex=False)

df['track_name'] = df['track_name'].apply(remove_accents)
df['artists'] = df['artists'].apply(remove_accents)

C:\Users\Destiny\AppData\Local\Temp\ipykernel_21844\1423931418.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,NaN,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,NaN,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


Build Artist level dataset 

- Call on API from The Audio DB to gain more information about the artist
- Use API to get Artist level instagram following information 

In [ ]:
'''
--------------------------------------
CREATE ARTIST DATAFRAME
--------------------------------------

Each row in our dataset is a song. The column Artists contains multiple artists who worked on a song. The code below will
split up the Artists (delimited by a semi-colan) and create a new dataframe that contains only unique artists.

This will be built up to create an artist level table, where we will add any additional characteristics about the artist that
could help predict song popularity.
'''
df_split = df['artists'].str.split(';', expand=True)
df_split.columns = [f'artist_{i+1}' for i in range(df_split.shape[1])]
df_final = pd.concat([df, df_split], axis=1)

unique_artists = df['artists'].str.split(';', expand=True).stack().unique()
unique_artists_df = pd.DataFrame(unique_artists, columns=['artists'])


In [6]:
'''
--------------------------------------
EDIT ORIGINAL DATASET TO BE AT SONG/ARTIST LEVEL
--------------------------------------
'''
# Ensure the 'artists' column is string type
df['artists'] = df['artists'].astype(str)

# Split the 'artists' column into lists where necessary
df['artists'] = df['artists'].apply(lambda x: x.split(';') if ';' in x else [x])

# Explode the 'artists' column to create one row per artist
df_exploded = df.explode('artists').reset_index(drop=True)

# Strip any leading/trailing whitespace from artist names
df_exploded['artists'] = df_exploded['artists'].str.strip()

print(df_exploded.head())

   Unnamed: 0                track_id            artists  \
0           0                     NaN        Gen Hoshino   
1           1  4qPNDBW1i3p13qLCt0Ki3A       Ben Woodward   
2           2  1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson   
3           2  1iJBSr7s7jYXzM8EGcbK5b               ZAYN   
4           3  6lfxq3CG4xtTiEg7opyCyx       Kina Grannis   

                                          album_name  \
0                                             Comedy   
1                                   Ghost (Acoustic)   
2                                     To Begin Again   
3                                     To Begin Again   
4  Crazy Rich Asians (Original Motion Picture Sou...   

                   track_name  popularity  duration_ms explicit  danceability  \
0                      Comedy          73       230666    False         0.676   
1            Ghost - Acoustic          55       149610      NaN         0.420   
2              To Begin Again          57       210826    F

In [ ]:
'''
---------------------------------
Create the below API call functions: 

API CALL 1: ARTIST ID FROM ARTIST NAME

API CALL 2: ARTIST DETAILS FROM ARTIST ID

API CALL 3: ARTIST SOCIAL MEDIA DATA FROM ARTIST ID
---------------------------------

Obtain Additional artist level data using The AudioDB 

NOTE: The API_KEY below will no longer be active. It was received from paying for a membership to The AudioDB which has since been cancelled.

'''

API_KEY = '523532'

#API CALL 1: Get Artist AudioDB ID using their name
def get_artist_info(artist_name):
    url = f"https://www.theaudiodb.com/api/v1/json/{API_KEY}/search.php?s={artist_name}"
    try:
        retry_count = 0
        while retry_count <10 :
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if 'artists' in data and data['artists'] is not None:
                    artist = data['artists'][0]
                    return artist.get('idArtist', 'N/A'), artist.get('strLabel', 'N/A')
                else:
                    #If response was successful but artist doesnt exist
                    return 'N/A','N/A'
            else:
                #If I get an error code
                print(f"Status Code: {response.status_code}")
                print(f"Response Headers: {response.headers}")
                data = response.json()
                print("Raw JSON Data:", data)
                retry_count+=1
                time.sleep(1)
    except Exception as e:
        print(f"Error fetching data for {artist_name}: {e}")
    return 'N/A', 'N/A'

#API CALL 2: Get Artist details by their AudioDB ID
def get_artist_details(artist_id):
    url = f"https://www.theaudiodb.com/api/v1/json/{API_KEY}/artist.php?i={artist_id}"
    try:
        retry_count = 0
        while retry_count <10 :
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if 'artists' in data and data['artists'] is not None:
                    artist = data['artists'][0]
                    return(
                        artist.get('strStyle', 'N/A'),
                        artist.get('strGenre', 'N/A'),
                        artist.get('strMood', 'N/A'),
                        artist.get('strFacebook', 'N/A'),
                        artist.get('strTwitter', 'N/A'),
                        artist.get('strGender', 'N/A'),
                        artist.get('strMusicBrainzID', 'N/A'),
                        artist.get('strLastFMChart', 'N/A')
                    )
                else:
                    #If response was successful but artist doesnt exist
                    return ('N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A')
            else:
                #If I get an error code
                print(f"Status Code: {response.status_code}")
                print(f"Response Headers: {response.headers}")
                data = response.json()
                print("Raw JSON Data:", data)
                retry_count+=1
                time.sleep(1)
    except Exception as e:
        print(f"Error fetching data for {artist_id}: {e}")
    return ('N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A')

#API CALL 3: Social media data:
def get_artist_socials(artist_id): 
    url = f"https://www.theaudiodb.com/api/v1/json/{API_KEY}/artist-links.php?i={artist_id}"
    
    #only want to pull info for these social medias:
    social_media = {
        'facebook': '',
        'twitter': '',
        'spotify': '',
        'deezer': '',
        'itunes': '',
        'tidal': '',
        'youtube': '',
        'instagram': ''
    }
    
    try:
        retry_count = 0
        while retry_count <10 :
            response = requests.get(url, timeout=10)
            #print(f"Received response. Status code: {response.status_code}") 

            if response.status_code == 200:
                data = response.json()
                #print(f"JSON data received: {data}")  # Debug print

                if 'artists' in data and data['artists'] is not None:
                    for link in data['artists']:
                        if link['strSite'] in social_media:
                            social_media[link['strSite']] = link['strLink']
                    return(social_media)
                else:
                    print(f"No social media data found for artist ID:  {artist_id}")
                    return (social_media)
            else:
                #If I get an error code
                print(f"Status Code: {response.status_code}")
                print(f"Response Headers: {response.headers}")
                data = response.json()
                print("Raw JSON Data:", data)
                retry_count+=1
                time.sleep(1)
    except Exception as e:
        print(f"Error fetching data for {artist_id}: {e}")
        
    #print(f"Returning social media data: {social_media}")  # Debug print
    return(social_media)

In [ ]:
'''
CALL API CALL FUNCTION 1 FROM ABOVE AND SAVE ARTIST LEVEL DATA

NOTE: This will not work without updating the API_KEY
'''

#Create two new columns to the artist data table
unique_artists_df['artist_id'] = [None] * len(unique_artists_df)
unique_artists_df['record_label'] = [None] * len(unique_artists_df)

# Apply the function to get data for two new columns
for index in range(len(unique_artists_df)):
    myartist = unique_artists_df['artists'][index]
    #print("processing:",myartist)
    callresponse = get_artist_info(myartist)
    unique_artists_df['artist_id'][index] = callresponse[0]
    unique_artists_df['record_label'][index] = callresponse[1]
    # Add a 1-second delay between API calls
    #time.sleep(1)

In [ ]:
'''
CALL API FUNCTION 2 FROM ABOVE AND SAVE ARTIST LEVEL DATA

NOTE: This will not work without updating the API_KEY
'''

unique_artists_details = unique_artists_df.copy()

#filter for only those with an audioDB id (to reduce run-time of the API call, we don't want to call on artists we know won't be in the database)
unique_artists_details = unique_artists_details[unique_artists_details['artist_id']!='N/A']

# Add new columns for the API results
unique_artists_details['strStyle'] = 'N/A'
unique_artists_details['strGenre'] = 'N/A'
unique_artists_details['strMood'] = 'N/A'
unique_artists_details['strFacebook'] = 'N/A'
unique_artists_details['strTwitter'] = 'N/A'
unique_artists_details['strGender'] = 'N/A'
unique_artists_details['strMusicBrainzID'] = 'N/A'
unique_artists_details['strLastFMChart'] = 'N/A'

# Apply the function to create two new columns
for index, row in unique_artists_details.iterrows():
    artist_id = row['artist_id']
    callresponse = get_artist_details(artist_id)
    
    # Update the DataFrame with API results
    unique_artists_details.at[index, 'strStyle'] = callresponse[0]
    unique_artists_details.at[index, 'strGenre'] = callresponse[1]
    unique_artists_details.at[index, 'strMood'] = callresponse[2]
    unique_artists_details.at[index, 'strFacebook'] = callresponse[3]
    unique_artists_details.at[index, 'strTwitter'] = callresponse[4]
    unique_artists_details.at[index, 'strGender'] = callresponse[5]
    unique_artists_details.at[index, 'strMusicBrainzID'] = callresponse[6]
    unique_artists_details.at[index, 'strLastFMChart'] = callresponse[7]

# Display the updated DataFrame & check we have all data
#print(len(unique_artists_details))
print(unique_artists_details.head(5))

             artists artist_id record_label          strStyle strGenre  \
2  Ingrid Michaelson    120114         None              Folk    Indie   
3               ZAYN    141655     Columbia          Rock/Pop      Pop   
4       Kina Grannis    111830         None          Rock/Pop      Pop   
6       Tyrone Wells    147082         None              None     Folk   
7  A Great Big World    126846         Epic  Manufactured Pop      Pop   

        strMood                     strFacebook strTwitter strGender  \
2                                                        1    Female   
3  Good Natured               facebook.com/zayn          1      Male   
4                en-gb.facebook.com/kinagrannis          1    Female   
6          None                            None       None      None   
7         Happy                                          1      Male   

                       strMusicBrainzID  \
2  1fc494a1-9109-4081-a455-2d05bea9d2bf   
3  985f7e6f-0a7e-4de7-b9ec-a5dac63cb

In [ ]:
'''
CALL API FUNCTION 3 ABOVE: Get artist social media data and add it to artist table

NOTE: This will not work without updating the API_KEY
'''

# Add new columns for the API results
unique_artists_details['twitter_link'] = [None] * len(unique_artists_details)
unique_artists_details['spotify_link'] = [None] * len(unique_artists_details)
unique_artists_details['deezer_link']  = [None] * len(unique_artists_details)
unique_artists_details['itunes_link']  = [None] * len(unique_artists_details)
unique_artists_details['tidal_link']   = [None] * len(unique_artists_details)
unique_artists_details['youtube_link'] = [None] * len(unique_artists_details)
unique_artists_details['instagram_link'] = [None] * len(unique_artists_details)


# Apply the function to create two new columns
for index, row in unique_artists_details.iterrows():

    artist_id = row['artist_id']
    callresponse = get_artist_socials(artist_id) #this is a dictionary now

    #print(callresponse)
    
    #loop through the dictionary:
    for platform, link in callresponse.items(): 
        # Update the DataFrame with API results
        unique_artists_details.at[index, f'{platform}_link'] = link
        #tester.at[index, f'{platform}_link'] = link

# Display the updated DataFrame & check we have all data
print(len(unique_artists_details))
print(unique_artists_details.head(5))

No social media data found for artist ID:  147082
No social media data found for artist ID:  177987
No social media data found for artist ID:  159291
No social media data found for artist ID:  133184
No social media data found for artist ID:  139910
No social media data found for artist ID:  145009
No social media data found for artist ID:  137006
No social media data found for artist ID:  133979
No social media data found for artist ID:  135073
No social media data found for artist ID:  136689
No social media data found for artist ID:  143164
No social media data found for artist ID:  165381
No social media data found for artist ID:  139688
No social media data found for artist ID:  157730
No social media data found for artist ID:  139503
No social media data found for artist ID:  179786
No social media data found for artist ID:  169341
No social media data found for artist ID:  162401
No social media data found for artist ID:  128427
No social media data found for artist ID:  153008


In [ ]:
'''
---------------------------------
Instagram follower API
---------------------------------
'''

def normalize_instagram_url(url):
    """
    Normalize Instagram URLs to ensure consistent matching
    - Remove protocol (http:// or https://)
    - Remove www. 
    - Remove trailing slashes
    - Convert to lowercase
    """
    if pd.isna(url):
        return ''
    
    # Ensure url is a string
    url = str(url)
    
    # Remove protocol
    url = re.sub(r'^https?://', '', url)
    
    # Remove www.
    url = re.sub(r'^www\.', '', url)
    
    # Remove trailing slash
    url = url.rstrip('/')
    
    # Convert to lowercase
    url = url.lower()
    
    return url

def merge_followers_count(db1_path, db2_path, output_path):
    try:
        # Read the CSV files
        db1 = pd.read_csv(db1_path, low_memory=False)
        db2 = pd.read_csv(db2_path, low_memory=False)
        
        # Normalize Instagram URLs in both dataframes
        db1['normalized_instagram_link'] = db1['instagram_link'].apply(normalize_instagram_url)
        db2['normalized_userUrl'] = db2['userUrl'].apply(normalize_instagram_url)
        
        # Print some debugging information
        print("Sample normalized links in DB1:")
        print(db1[['instagram_link', 'normalized_instagram_link']].head())
        
        print("\nSample normalized links in DB2:")
        print(db2[['userUrl', 'normalized_userUrl']].head())
        
        # Create a copy of db1
        merged_db = db1.copy()
        
        # Create a new column for followers count
        merged_db['instagram_followers_count'] = ''
        
        # Create a dictionary to map normalized Instagram URLs to followers count
        followers_dict = dict(zip(db2['normalized_userUrl'], db2['followersCount']))
        
        # Debugging: Print dictionary size and some entries
        print(f"\nTotal unique normalized URLs in followers dictionary: {len(followers_dict)}")
        print("Sample dictionary entries:")
        for k, v in list(followers_dict.items())[:5]:
            print(f"{k}: {v}")
        
        # Counters for tracking matches
        total_urls = 0
        matched_urls = 0
        
        # Iterate through the rows and update followers count
        for index, row in merged_db.iterrows():
            # Get the normalized Instagram link
            normalized_link = row['normalized_instagram_link']
            
            # Skip if no link
            if not normalized_link:
                continue
            
            total_urls += 1
            
            # Try to find a match in the followers dictionary
            followers_count = followers_dict.get(normalized_link, '')
            
            if followers_count:
                matched_urls += 1
                merged_db.at[index, 'instagram_followers_count'] = followers_count
        
        # Print matching statistics
        print(f"\nTotal Instagram URLs processed: {total_urls}")
        print(f"Matched URLs: {matched_urls}")
        print(f"Matching percentage: {matched_urls/total_urls*100:.2f}%")
        
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        merged_db.to_csv(output_path, index=False)
        
        print(f"\nMerged database saved to {output_path}")
        
        return merged_db
    
    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()
        return None

if __name__ == "__main__":
    merged_result = merge_followers_count(
        r'original_db_additional_columns(in).csv',
        r'insta_followers.csv',
        r'A:\new DB\merged_database.csv'
    )


Build Song level data 
- Use The Audio DB API
- Check if the song has trended on the app TikTok

In [ ]:
'''
---------------------------------
Joining initial Spotify and Tiktok datasets

---------------------------------
'''
def join_spotify_tiktok_datasets(spotify_file_path, tiktok_file_path, output_file_path=None):
    """
    Join Spotify and TikTok datasets based on track_id.
    
    Parameters:
    -----------
    spotify_file_path : str
        Path to the Spotify dataset CSV file
    tiktok_file_path : str
        Path to the TikTok dataset CSV file
    output_file_path : str, optional
        Path to save the joined dataset. If None, the function will return the DataFrame
        
    Returns:
    --------
    pd.DataFrame
        The joined dataset if output_file_path is None, otherwise None
    """
    # Load the datasets
    print(f"Loading Spotify dataset from {spotify_file_path}...")
    spotify_df = pd.read_csv(spotify_file_path)
    
    print(f"Loading TikTok dataset from {tiktok_file_path}...")
    tiktok_df = pd.read_csv(tiktok_file_path)
    
    # Clean up the datasets
    if '' in spotify_df.columns:
        spotify_df = spotify_df.drop('', axis=1)
    
    # Rename columns to avoid conflicts and clearly identify the source
    common_columns = set(spotify_df.columns).intersection(set(tiktok_df.columns))
    common_columns.discard('track_id')  # Keep track_id unchanged as it's our join key
    
    # Rename common columns in TikTok dataset
    rename_dict = {col: f'tiktok_{col}' for col in common_columns}
    tiktok_df = tiktok_df.rename(columns=rename_dict)
    
    # Rename common columns in Spotify dataset
    rename_dict = {col: f'spotify_{col}' for col in common_columns}
    spotify_df = spotify_df.rename(columns=rename_dict)
    
    # Handle missing track_ids (if any)
    spotify_df = spotify_df.dropna(subset=['track_id'])
    tiktok_df = tiktok_df.dropna(subset=['track_id'])
    
    # Perform the join
    print("Joining datasets on track_id...")
    joined_df = pd.merge(
        spotify_df, 
        tiktok_df, 
        on='track_id', 
        how='outer',  
        indicator=True  
    )
    
    # Print some info about the join
    merge_counts = joined_df['_merge'].value_counts()
    print("\nJoin Results:")
    print(f"Total rows in joined dataset: {len(joined_df)}")
    print(f"Rows only in Spotify: {merge_counts.get('left_only', 0)}")
    print(f"Rows only in TikTok: {merge_counts.get('right_only', 0)}")
    print(f"Rows in both datasets: {merge_counts.get('both', 0)}")
    
    # Save to file if output path is provided
    if output_file_path:
        print(f"Saving joined dataset to {output_file_path}...")
        joined_df.to_csv(output_file_path, index=False)
        print("Done!")
        return None
    
    return joined_df

if __name__ == "__main__":
    # Update these paths to match your file locations
    spotify_path = "Spotify Datasetin.csv"
    tiktok_path = "tiktok.csv"
    output_path = "joined_spotify_tiktok.csv"
    
    join_spotify_tiktok_datasets(spotify_path, tiktok_path, output_path)
    

In [ ]:
'''
---------------------------------
Create the below API call function to get TRACK/SONG level data: 

API CALL 1: TRACK INFO FROM ARTIST NAME & TRACK NAME

'''
def get_track_details(artist_name, track_name):
    url = f"https://www.theaudiodb.com/api/v1/json/{API_KEY}/searchtrack.php?s={artist_name}&t={track_name}"
    try:
        retry_count = 0
        while retry_count <10 :
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if 'track' in data and data['track'] is not None:
                    track = data['track'][0]
                    return(
                        track.get('idTrack', 'N/A'),
                        track.get('strTheme', 'N/A'),
                        track.get('strMusicVid', 'N/A'),
                        track.get('intMusicVidViews', 'N/A'),
                        track.get('intMusicVidLikes', 'N/A'),
                        track.get('intMusicVidDislikes', 'N/A'),
                        track.get('intMusicVidComments', 'N/A'),
                        track.get('intTotalListeners', 'N/A'),
                        track.get('intTotalPlays', 'N/A'),
                        track.get('strMusicBrainzID', 'N/A') #probably dont need but just in case
                    )
                else:
                    #If response was successful but artist doesnt exist
                    # return ('N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A')
                    return (None, None, None, None, None, None, None, None, None, None)
            else:
                #If I get an error code
                print(f"Status Code: {response.status_code}")
                print(f"Response Headers: {response.headers}")
                data = response.json()
                print("Raw JSON Data:", data)
                retry_count+=1
                time.sleep(1)
    except Exception as e:
        print(f"Error fetching data for {artist_name} and {track_name}: {e}")
    # return ('N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A')
    return (None, None, None, None, None, None, None, None, None, None)



In [8]:
'''This code chunk was optimized to run faster in the chunk below'''

# '''Add Track level data'''

# #Create new columns to the artist data table
# df_exploded['idTrack']      = [None] * len(df_exploded)
# df_exploded['strTheme']     = [None] * len(df_exploded)
# df_exploded['strMusicVid']  = [None] * len(df_exploded)
# df_exploded['intMusicVidViews']  = [None] * len(df_exploded)
# df_exploded['intMusicVidLikes']  = [None] * len(df_exploded)
# df_exploded['intMusicVidDislikes'] = [None] * len(df_exploded)
# df_exploded['intMusicVidComments'] = [None] * len(df_exploded)
# df_exploded['intTotalListeners']   = [None] * len(df_exploded)
# df_exploded['intTotalPlays'] = [None] * len(df_exploded)
# df_exploded['strMusicBrainzID'] = [None] * len(df_exploded)

# # Apply the function to get data for two new columns
# for index, row in df_exploded.iterrows():
#     artistname = df_exploded['artists'][index]
#     trackname = df_exploded['track_name'][index]

#     #print("processing:",artistname,",",trackname)
#     if index % 1000 == 0:
#         print(f"Reached the {index}th iteration")
#     callresponse = get_track_details(artistname, trackname)
#     df_exploded.loc[index, 'idTrack']        = callresponse[0]
#     df_exploded.loc[index, 'strTheme']       = callresponse[1]
#     df_exploded.loc[index, 'strMusicVid']    = callresponse[2]
#     df_exploded.loc[index, 'intMusicVidViews']    = callresponse[3]
#     df_exploded.loc[index, 'intMusicVidLikes']    = callresponse[4]
#     df_exploded.loc[index, 'intMusicVidDislikes'] = callresponse[5]
#     df_exploded.loc[index, 'intMusicVidComments'] = callresponse[6]
#     df_exploded.loc[index, 'intTotalListeners']   = callresponse[7]
#     df_exploded.loc[index, 'intTotalPlays']       = callresponse[8]
#     df_exploded.loc[index, 'strMusicBrainzID']    = callresponse[9]

#     # Add a 1-second delay between API calls
#     #time.sleep(1)

'This code chunk was optimized to run faster in the chunk below'

In [ ]:
'''Add Track Level Data'''
#same as above code chunk. Just updated to run faster (SRC: https://www.digitalocean.com/community/tutorials/how-to-use-threadpoolexecutor-in-python-3)

# Add new columns
columns_to_add = [
    'idTrack', 'strTheme', 'strMusicVid', 'intMusicVidViews', 'intMusicVidLikes',
    'intMusicVidDislikes', 'intMusicVidComments', 'intTotalListeners',
    'intTotalPlays', 'strMusicBrainzID'
]
for col in columns_to_add:
    df_exploded[col] = None

# Function wrapper to include index for mapping back
def fetch_track_data(index, artistname, trackname):
    try:
        result = get_track_details(artistname, trackname)
        return index, result
    except Exception as e:
        print(f"Error at index {index}: {e}")
        return index, [None] * 10  # output if error

# Prepare input list
inputs = [(index, row['artists'], row['track_name']) for index, row in df_exploded.iterrows()]

# Run in batches of 10
batch_size = 10
for i in range(0, len(inputs), batch_size):

    print(f"Processing batch {i} to {i+batch_size}")
    batch = inputs[i:i + batch_size]

    with ThreadPoolExecutor(max_workers=batch_size) as executor:
        futures = [executor.submit(fetch_track_data, *args) for args in batch]

        for future in as_completed(futures):
            index, data = future.result()
            if data:
                for col, value in zip(columns_to_add, data):
                    df_exploded.at[index, col] = value


Processing batch 0 to 10
Processing batch 10 to 20
Processing batch 20 to 30
Processing batch 30 to 40
Processing batch 40 to 50
Processing batch 50 to 60
Processing batch 60 to 70
Processing batch 70 to 80
Processing batch 80 to 90
Processing batch 90 to 100
Processing batch 100 to 110
Processing batch 110 to 120
Processing batch 120 to 130
Processing batch 130 to 140
Processing batch 140 to 150
Processing batch 150 to 160
Processing batch 160 to 170
Processing batch 170 to 180
Processing batch 180 to 190
Processing batch 190 to 200
Processing batch 200 to 210
Processing batch 210 to 220
Processing batch 220 to 230
Processing batch 230 to 240
Processing batch 240 to 250
Processing batch 250 to 260
Processing batch 260 to 270
Processing batch 270 to 280
Processing batch 280 to 290
Processing batch 290 to 300
Processing batch 300 to 310
Processing batch 310 to 320
Processing batch 320 to 330
Processing batch 330 to 340
Processing batch 340 to 350
Processing batch 350 to 360
Processing b

In [ ]:
'''Validate the track level data pull from cell above'''

#de_dup_tracks.to_csv('track_data.csv')
#de_dup_tracks[(de_dup_tracks['strMusicVid'] != 'N/A') & (de_dup_tracks['strMusicVid'].notna())].head(5)
de_dup_tracks.rename(columns={'Track_MusicBrainzID_y': 'Track_MusicBrainzID'}, inplace=True)
len(de_dup_tracks)


'''Remove any duplicates from both dataframes'''

# Get all column names except the index
columns_to_check = [col for col in df_exploded.columns if col != 'Unnamed:']

# Drop duplicates based on all columns except the excluded one
de_dup_tracks = df_exploded.drop_duplicates(subset=columns_to_check)

Clean up the Song level dataset

In [ ]:
#Fix the null values to be consistent. Instead of 'N/A', it can just be None
de_dup_tracks['Track_MusicVidViews'] = de_dup_tracks['Track_MusicVidViews'].apply(lambda x: None if x == "N/A" else x)
de_dup_tracks['Track_MusicVidLikes'] = de_dup_tracks['Track_MusicVidLikes'].apply(lambda x: None if x == "N/A" else x)
de_dup_tracks['Track_MusicVidDislikes'] = de_dup_tracks['Track_MusicVidDislikes'].apply(lambda x: None if x == "N/A" else x)
de_dup_tracks['Track_MusicVidComments'] = de_dup_tracks['Track_MusicVidComments'].apply(lambda x: None if x == "N/A" else x)
de_dup_tracks['Track_totalListeners'] = de_dup_tracks['Track_totalListeners'].apply(lambda x: None if x == "N/A" else x)
de_dup_tracks['Track_TotalPlays'] = de_dup_tracks['Track_TotalPlays'].apply(lambda x: None if x == "N/A" else x)

print(len(de_dup_tracks))

#de_dup_tracks.to_csv('Final_Track_DB_V1.csv')

158293


Below, we try adding another predictor variable.

We will check to see if the song has appeared on the Billboard Hot 100 charts.

We will test our model's ability to predict both the popularity score that is within the original dataset, as well as the below created column: artist_and_song_in_hot100 

Source of hot 100 data: https://github.com/HipsterVizNinja/random-data/tree/main/Music/hot-100


In [ ]:
'''Add column for Billboard Hot 100'''

file_path = 'Hot 100.csv'
hot100 = pd.read_csv(file_path)

In [ ]:
'''To add additional fields, we can use the code below to upload our previously saved file and add onto it. 

Commenting it out for readability. This does not need to be used if running the code in order. It is the same as the first cells ran in the beginning of this program.'''

# de_dup_tracks = pd.read_csv('Final_Track_DB_1.csv', encoding="utf-8")

# #clean table: if track name contains "(feat. %)" or "(with %)" then remove that text from the song title 
# #using regular expressions and making sure its not case sensitive
# de_dup_tracks['track_name'] = de_dup_tracks['track_name'].replace(r'(?i)\s*\(feat\.?[^)]*\)','', regex=True)
# de_dup_tracks['track_name'] = de_dup_tracks['track_name'].replace(r'(?i)\s*\(with\.?[^)]*\)','', regex=True)

# #fix text issue i am seeing
# def fix_text(text):
#     if isinstance(text, str):
#         try:
#             # Try cp1252 encoding with replacing problematic bytes,
#             # then decoding as utf-8 while replacing problematic bytes
#             return text.encode('cp1252', errors='replace').decode('utf-8', errors='replace')
#         except UnicodeDecodeError:
#             # Fallback: ignore decode errors if the above fails
#             return text.encode('cp1252', errors='replace').decode('utf-8', errors='ignore')
#     return text

# import unicodedata

# #to match hot100, it doesnt have the spanish alphabet characters. the code below should normalize the song titles to match the database
# def remove_accents(text):
#     text = str(text) #some songs are numbers (:
#     # Normalize the text to decompose accented characters
#     normalized_text = unicodedata.normalize('NFKD', text)
#     # Remove the combining characters (accents) by filtering them out
#     return "".join(c for c in normalized_text if not unicodedata.combining(c))

# de_dup_tracks['track_name'] = de_dup_tracks['track_name'].apply(fix_text)
# de_dup_tracks['artists'] = de_dup_tracks['artists'].apply(fix_text)

# de_dup_tracks['track_name'] = de_dup_tracks['track_name'].str.replace("’", "'", regex=False)
# de_dup_tracks['artists'] = de_dup_tracks['artists'].str.replace("’", "'", regex=False)

# de_dup_tracks['track_name'] = de_dup_tracks['track_name'].apply(remove_accents)
# de_dup_tracks['artists'] = de_dup_tracks['artists'].apply(remove_accents)


C:\Users\Destiny\AppData\Local\Temp\ipykernel_14872\930603557.py:1: DtypeWarning: Columns (10,45,49,50,51,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  de_dup_tracks = pd.read_csv('Final_Track_DB_1.csv', encoding="utf-8")


In [ ]:
'''The column for artists in the hot100 dataset is delimited with many different methods: "&", ",", "featuring", "feat.", "with", etc.  
so we need to split up the hot100 dataset to be at a song/artist level like our current dataset. However, sometimes one of these 
characters is valid in the name like "Tyler, The Creator". So, we can check if the song exists since we know our original dataset is
delimited with ";" we can check if the song exists in the billboard hot 100, and if it does, we can check if the artist in our original
dataframe is contained within the performers column of the billboard data '''

import string

#Function to make text comparable
def normalize(text):
    if pd.isna(text):
        return ""
    return str(text).strip().lower()

#Use function to level out our comparison
de_dup_tracks['track_name'] = de_dup_tracks['track_name'].apply(normalize)
de_dup_tracks['artists']    = de_dup_tracks['artists'].apply(normalize)
hot100['song'] = hot100['song'].apply(normalize)
hot100['performer'] = hot100['performer'].apply(normalize)

#Validate it is working properly:
#print("hot 100:")
#print(hot100[hot100['song'].str.contains("i don't wanna live forever")].head(5))

#print("df before adding new col:")
#print(de_dup_tracks[(de_dup_tracks['artists'].apply(normalize).str.contains('taylor swift'))])
#print(de_dup_tracks[de_dup_tracks['track_name'].str.contains("i don't wanna live forever")])

#Check if song exists in Hot100
de_dup_tracks['song_exists_in_hot100'] = de_dup_tracks['track_name'].isin(hot100['song'])

#Validate what song exists in hot 100 shows
#print("df after adding new col")
#print(de_dup_tracks[(de_dup_tracks['artists'].apply(normalize).str.contains('taylor swift'))])
#print(de_dup_tracks[de_dup_tracks['track_name'].str.contains("i don't wanna live forever")])

# Group all performers for a song and joins them with a space, so everyone is one string of text.
hot100_grouped = hot100.groupby('song')['performer'].apply(lambda x: ' '.join(x)).reset_index()

# Create a lookup dictionary from the grouped DataFrame
hot100_lookup = hot100_grouped.set_index('song')['performer'].to_dict()

#check if the artist form our original dataframe is within the hot100
def robust_check_artist(artist, performer):
    # Remove punctuation and whitespace to compare
    translator = str.maketrans('', '', string.punctuation)
    artist_clean = artist.translate(translator).replace(" ", "")
    performer_clean = performer.translate(translator).replace(" ", "")
    return artist_clean in performer_clean

def check_artist_for_song(row):
    song = row['track_name']
    artist = row['artists']
    performer = hot100_lookup.get(song, "")
    return robust_check_artist(artist, performer)

#Apply the check with new column
de_dup_tracks['artist_and_song_in_hot100'] = de_dup_tracks.apply(check_artist_for_song, axis=1)

#Check if there are mismatches between the song match and song+artist match (this is expected, but can be manually reviewed to ensure the values make sense)
debug_mismatch = pd.merge(
    de_dup_tracks[ (de_dup_tracks['song_exists_in_hot100']) & (~de_dup_tracks['artist_and_song_in_hot100']) ],
    hot100_grouped,
    left_on='track_name',
    right_on='song',
    how='left',
    suffixes=('_df1', '_hot100')
)

debug_mismatch = debug_mismatch[[
    'track_name',     # normalized song from de_dup_tracks
    'artists',        # normalized artist from de_dup_tracks
    'song_exists_in_hot100',
    'artist_and_song_in_hot100',
    'song',           # normalized song from hot100_grouped
    'performer'       # aggregated performer(s) from hot100
]]

In [164]:
'''Validate the new flags created for hot 100

which songs are mismatched between the two methods and are they valid?'''


print("Length of mismatch df: ",len(debug_mismatch))
print("how many songs/artists ARE in the hot 100: ", len(de_dup_tracks[de_dup_tracks['artist_and_song_in_hot100']==True]))

#Manually validate specific examples for high popularity artists: 
# sorted_df = de_dup_tracks[
#     (de_dup_tracks['artists'] == 'bad bunny') & 
#     (de_dup_tracks['artist_and_song_in_hot100'] != True)
# ].sort_values(by='popularity', ascending=False)

#Validate that any songs showing as a mismatch are not an error. (we would expect high popularity to be more likely to be in the hot 100)
sorted_df = de_dup_tracks[
    (de_dup_tracks['song_exists_in_hot100'] == True ) &
    (de_dup_tracks['artist_and_song_in_hot100'] != True)
].sort_values(by='popularity', ascending=False)

sorted_df.head(10)

Length of mismatch df:  16711
how many songs/artists ARE in the hot 100:  9650


,Row,track_id,artists,album_name,track_name,Unnamed: 5,Unnamed: 6,concat,popularity,duration_ms,...,Track_MusicVidViews,Track_MusicVidLikes,Track_MusicVidDislikes,Track_MusicVidComments,Track_totalListeners,Track_TotalPlays,Track_MusicBrainzID,Hot100,song_exists_in_hot100,artist_and_song_in_hot100
113206,81024,0mBP9X2gPCuapvpZ7TGDk3,bts,Left and Right (Feat. Jung Kook of BTS),left and right,NaN,NaN,810240mBP9X2gPCuapvpZ7TGDk3BTSLeft and Right (...,92,154486,...,0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False
25985,20017,0mBP9X2gPCuapvpZ7TGDk3,bts,Left and Right (Feat. Jung Kook of BTS),left and right,NaN,NaN,200170mBP9X2gPCuapvpZ7TGDk3BTSLeft and Right (...,92,154486,...,0,0.0,NaN,NaN,NaN,NaN,NaN,False,True,False
48519,34014,1Fid2jjqsHViMX6xNH70hE,seafret,Tell Me It's Real (Expanded Edition),atlantis,NaN,NaN,340141Fid2jjqsHViMX6xNH70hESeafretTell Me It's...,90,229173,...,0,0.0,0.0,0.0,56922.0,373887.0,f6471913-5792-4046-946c-0f4447a1c543,False,True,False
114663,81857,1R0a2iXumgCiFb7HEZ7gUE,taylor swift,reputation,don't blame me,NaN,NaN,818571R0a2iXumgCiFb7HEZ7gUETaylor Swiftreputat...,88,236413,...,0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False
120152,86000,7EkWXAI1wn8Ii883ecd9xr,surf curse,Freaks,freaks,NaN,NaN,860007EkWXAI1wn8Ii883ecd9xrSurf CurseFreaksFreaks,86,147062,...,0,0.0,0.0,0.0,0.0,0.0,63180294-3515-45e8-b418-72d04597ca35,False,True,False
71710,51503,4woTEX1wYOTGDqNXuavlRC,dina rae,The Eminem Show,superman,NaN,NaN,515034woTEX1wYOTGDqNXuavlRCDina RaeThe Eminem ...,86,350320,...,0,0,NaN,NaN,NaN,NaN,NaN,False,True,False
89712,65056,0RDqNCRBGrSegk16Avfzuq,twice,BETWEEN 1&2,talk that talk,NaN,NaN,650560RDqNCRBGrSegk16AvfzuqTWICEBETWEEN 1&2Tal...,86,177466,...,0,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False
80376,57054,7EkWXAI1wn8Ii883ecd9xr,surf curse,Freaks,freaks,NaN,NaN,570547EkWXAI1wn8Ii883ecd9xrSurf CurseFreaksFreaks,86,147062,...,0,0,0,0,0,0,63180294-3515-45e8-b418-72d04597ca35,False,True,False
79098,56051,7EkWXAI1wn8Ii883ecd9xr,surf curse,Freaks,freaks,NaN,NaN,560517EkWXAI1wn8Ii883ecd9xrSurf CurseFreaksFreaks,86,147062,...,0,0,0,0,0,0,63180294-3515-45e8-b418-72d04597ca35,False,True,False
42534,30365,1bgKMxPQU7JIZEhNsM1vFs,zara larsson,Words (feat. Zara Larsson),words,NaN,NaN,303651bgKMxPQU7JIZEhNsM1vFsZara LarssonWords (...,85,142677,...,0,0.0,NaN,NaN,NaN,NaN,NaN,False,True,False


In [ ]:
de_dup_tracks.to_csv('Final_Track_DB_V3.csv')